In [1]:
from typing import List

import pandas as pd
import numpy as np
import pglast

import logparsing
import action_selection
import action_generation.index_actions as index_actions
import db_connector

In [2]:
[str(a) for a in index_actions.DropIndexGenerator(db_connector.get_existing_indexes())]

['DROP INDEX IF EXISTS idx_jungle_intfield3_floatfield7;',
 'DROP INDEX IF EXISTS idx_jungle_intfield7;',
 'DROP INDEX IF EXISTS idx_jungle_uuidfield;']

In [3]:
workload_csv = "artifacts/indexjungle/workload.csv"
timeout = 100

In [4]:
col_mappings = db_connector.get_col_mappings()
parsed = logparsing.parse_csv_log(workload_csv)
filtered = logparsing.aggregate_templates(parsed, col_mappings, .9999)
colrefs = action_selection.get_workload_colrefs(filtered)

In [4]:
filtered['percentage'] = filtered['count']/filtered['count'].sum()
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
filtered.to_csv("test.csv")

In [5]:
action_selection.generate_sql(workload_csv, None)

Estimating pre-hypothetical costs
cost estimate iter: 0
cost estimate iter: 1
cost estimate iter: 2
cost estimate iter: 3
cost estimate iter: 4
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_intfield6 ON jungle (int_field6);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_intfield2 ON jungle (int_field2);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_intfield8 ON jungle (int_field8);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield5 ON jungle (float_field5);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield8 ON jungle (float_field8);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield2 ON jungle (float_field2);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield1 ON jungle (float_field1);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield0 ON jungle (float_field0);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_jungle_floatfield4 ON jungle (float_field4);
Creating hypo

/home/mkpjnx/repos/15799-index-picker/action_selection.py:104: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [26]:
db_connector.get_existing_indexes()

[('jungle',
  'idx_jungle_intfield3_floatfield7',
  'CREATE INDEX idx_jungle_intfield3_floatfield7 ON public.jungle USING btree (int_field3, float_field7)',
  ['int_field3', 'float_field7']),
 ('jungle',
  'idx_jungle_intfield7',
  'CREATE INDEX idx_jungle_intfield7 ON public.jungle USING btree (int_field7)',
  ['int_field7']),
 ('jungle',
  'idx_jungle_uuidfield',
  'CREATE INDEX idx_jungle_uuidfield ON public.jungle USING btree (uuid_field)',
  ['uuid_field'])]

In [10]:
gen = index_actions.ExhaustiveIndexGenerator(colrefs,2)

In [6]:
db_connector.get_unused_indexes()

[('jungle', 'idx_jungle_uuidfield', 606208),
 ('jungle', 'idx_jungle_intfield3_floatfield7', 335872),
 ('jungle', 'idx_jungle_intfield7', 245760)]

In [7]:
db_connector.get_existing_indexes()

[('jungle',
  'idx_jungle_intfield3_floatfield7',
  'CREATE INDEX idx_jungle_intfield3_floatfield7 ON public.jungle USING btree (int_field3, float_field7)',
  ['int_field3', 'float_field7'],
  335872),
 ('jungle',
  'idx_jungle_intfield7',
  'CREATE INDEX idx_jungle_intfield7 ON public.jungle USING btree (int_field7)',
  ['int_field7'],
  245760),
 ('jungle',
  'idx_jungle_uuidfield',
  'CREATE INDEX idx_jungle_uuidfield ON public.jungle USING btree (uuid_field)',
  ['uuid_field'],
  606208)]